Import packages

In [1]:
import torch
from torch import optim
import sys
sys.path.append('../..')
from nflows.flows.base import Flow
from nflows.distributions.normal import ConditionalDiagonalNormal, StandardNormal
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedAffineAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.base import CompositeTransform
import numpy as np

import torch

from torch.utils.tensorboard import SummaryWriter

import scripts.utils as utils

Import data

In [2]:
train_y, train_x = utils.get_training_data()
test_y, test_x = utils.get_test_data()
train_y = utils.normalize_numpy(train_y, False, True)
test_y = utils.normalize_numpy(test_y, False, True)
train_y = torch.tensor(train_y).float()
train_x = torch.tensor(train_x).float()
test_y = torch.tensor(test_y).float()
test_x = torch.tensor(test_x).float()

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

print(train_x.shape,train_y.shape,test_x.shape,test_y.shape)

torch.Size([45656, 10]) torch.Size([45656, 96]) torch.Size([11414, 10]) torch.Size([11414, 96])


Parameter setup

In [4]:
epochs = 50000

num_layers = 5
base_dist = StandardNormal([96])
#base_dist = ConditionalDiagonalNormal([96],context_encoder=model)
transforms = []

for _ in range(num_layers):
    transforms.append(ReversePermutation(features=96))
    transforms.append(MaskedAffineAutoregressiveTransform(features=96,
                                                          hidden_features=256,
                                                          context_features=10))
    
transform = CompositeTransform(transforms)
flow = Flow(transform, base_dist).cuda()
optimizer = optim.Adam(flow.parameters())

Training

In [5]:
import os
run = 'run3'
output_path = r'C:\Thesis\scripts\normalizing-flows\runs'
writer = SummaryWriter(os.path.join(output_path,run))

for i in range(epochs):
    optimizer.zero_grad()
    loss = -flow.log_prob(inputs=train_y, context=train_x).mean()
    test_loss = -flow.log_prob(inputs=test_y, context=test_x).mean()
    writer.add_scalar("Loss/train",loss,i)
    writer.add_scalar("Loss/test",test_loss,i)
    if i%100 == 0:
      print('iteration', i,':',loss.item(),':',test_loss.item())
      output_model = os.path.join(output_path,run,f'model-{i}')
      torch.save(flow,output_model)
    loss.backward()
    optimizer.step()

iteration 0 : 262.7591552734375 : 262.76025390625
iteration 100 : -182.32083129882812 : 58.32097244262695
iteration 200 : -221.04750061035156 : 29.12255859375
iteration 300 : -245.12103271484375 : -46.42838668823242
iteration 400 : -238.00796508789062 : -46.597084045410156
iteration 500 : -250.22068786621094 : 274.6990966796875
iteration 600 : -234.407958984375 : -50.74391174316406
iteration 700 : -255.08331298828125 : -9.94276237487793
iteration 800 : -269.9414367675781 : -34.461421966552734
iteration 900 : -274.62890625 : -52.26594543457031
iteration 1000 : -263.5885314941406 : -71.35167694091797
iteration 1100 : -274.20623779296875 : -63.89247131347656
iteration 1200 : -276.9622497558594 : -40.918209075927734
iteration 1300 : -273.0816650390625 : -21.534099578857422
iteration 1400 : -278.00274658203125 : 170.95831298828125
iteration 1500 : -277.7588806152344 : 1047.81640625
iteration 1600 : -283.6842041015625 : 383.7525939941406
iteration 1700 : -289.914306640625 : 4148.45361328125


KeyboardInterrupt: 